In [ ]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from imblearn.over_sampling import SMOTE

from google.colab import files
uploaded = files.upload()




KeyboardInterrupt: 

In [ ]:

df = pd.read_csv("cleaned_health_dataset.csv")


In [ ]:
df['exercise'] = (
    df['exercise']
    .astype(str)
    .str.strip()
    .str.lower()
    .replace('none', 'no exercise')
)

print(df['exercise'].unique())





['low' 'high' 'medium' 'no exercise']


In [ ]:
df


,age,weight,height,exercise,sleep,sugar_intake,smoking,alcohol,profession,health_risk,bmi_recalc
0,56,67,195,low,6.1,medium,yes,yes,office_worker,high,17.619987
1,69,76,170,high,6.9,high,no,no,teacher,high,26.297578
2,46,106,153,high,6.6,low,yes,no,artist,high,45.281729
3,32,54,186,medium,8.5,medium,no,no,artist,low,15.608741
4,60,98,195,high,8.0,low,no,no,teacher,high,25.772518
...,...,...,...,...,...,...,...,...,...,...,...
4993,42,75,187,medium,7.2,low,no,no,driver,low,21.447568
4994,39,86,183,low,7.4,low,no,no,engineer,high,25.680074
4995,48,56,194,no exercise,7.2,high,yes,yes,engineer,high,14.879371
4996,34,109,148,medium,9.9,medium,no,no,doctor,low,49.762600


In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Convert both columns to lowercase
df['exercise'] = df['exercise'].str.lower().str.strip()
df['sugar_intake'] = df['sugar_intake'].str.lower().str.strip()

# Define correct lowercase category order
exercise_order = ['no exercise', 'low', 'medium', 'high']
sugar_order = ['low', 'medium', 'high']

# Apply ordinal encoding
ordinal = OrdinalEncoder(categories=[exercise_order, sugar_order])
df[['exercise', 'sugar_intake']] = ordinal.fit_transform(df[['exercise', 'sugar_intake']])

print(df[['exercise', 'sugar_intake']].head())


   exercise  sugar_intake
0       1.0           1.0
1       3.0           2.0
2       3.0           0.0
3       2.0           1.0
4       3.0           0.0


In [ ]:
# 4️⃣ One-Hot Encoding for 'smoking', 'alcohol', 'healthrisk'
onehot = pd.get_dummies(df[['smoking', 'alcohol', 'health_risk']], drop_first=True)

In [ ]:
# Drop original columns and concatenate the encoded ones
df = df.drop(['smoking', 'alcohol', 'health_risk'], axis=1)
df = pd.concat([df, onehot], axis=1)

In [ ]:
df.head()


,age,weight,height,exercise,sleep,sugar_intake,profession,bmi_recalc,smoking_yes,alcohol_yes,health_risk_low
0,56,67,195,1.0,6.1,1.0,office_worker,17.619987,True,True,False
1,69,76,170,3.0,6.9,2.0,teacher,26.297578,False,False,False
2,46,106,153,3.0,6.6,0.0,artist,45.281729,True,False,False
3,32,54,186,2.0,8.5,1.0,artist,15.608741,False,False,True
4,60,98,195,3.0,8.0,0.0,teacher,25.772518,False,False,False


In [ ]:
df['health_risk_high'] = 1 - df['health_risk_low']
df = df.drop(columns=['health_risk_low'])


In [ ]:
df.head()

,age,weight,height,exercise,sleep,sugar_intake,profession,bmi_recalc,smoking_yes,alcohol_yes,health_risk_high
0,56,67,195,1.0,6.1,1.0,office_worker,17.619987,True,True,1
1,69,76,170,3.0,6.9,2.0,teacher,26.297578,False,False,1
2,46,106,153,3.0,6.6,0.0,artist,45.281729,True,False,1
3,32,54,186,2.0,8.5,1.0,artist,15.608741,False,False,0
4,60,98,195,3.0,8.0,0.0,teacher,25.772518,False,False,1


In [ ]:
df = pd.get_dummies(df, columns=['profession'], drop_first=True)


In [ ]:
df.head()

,age,weight,height,exercise,sleep,sugar_intake,bmi_recalc,smoking_yes,alcohol_yes,health_risk_high,profession_doctor,profession_driver,profession_engineer,profession_farmer,profession_office_worker,profession_student,profession_teacher
0,56,67,195,1.0,6.1,1.0,17.619987,True,True,1,False,False,False,False,True,False,False
1,69,76,170,3.0,6.9,2.0,26.297578,False,False,1,False,False,False,False,False,False,True
2,46,106,153,3.0,6.6,0.0,45.281729,True,False,1,False,False,False,False,False,False,False
3,32,54,186,2.0,8.5,1.0,15.608741,False,False,0,False,False,False,False,False,False,False
4,60,98,195,3.0,8.0,0.0,25.772518,False,False,1,False,False,False,False,False,False,True


In [ ]:
# ==== STEP 0: Imports ====
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# ==== STEP 1: Verify your target column ====
# (Should already exist as 0/1 integers)
print(df['health_risk_high'].value_counts())

health_risk_high
1    3490
0    1508
Name: count, dtype: int64


In [ ]:
# ==== STEP 2: Split features (X) and target (y) ====
X = df.drop(columns=['health_risk_high'])
y = df['health_risk_high']

# ==== STEP 3: Train/test split (keep same ratio using stratify) ====
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
print("Class distribution before balancing:")
print("Train set:\n", y_train.value_counts())
print("Test set:\n", y_test.value_counts())


Class distribution before balancing:
Train set:
 health_risk_high
1    2792
0    1206
Name: count, dtype: int64
Test set:
 health_risk_high
1    698
0    302
Name: count, dtype: int64


In [ ]:
# ==== STEP 4: Apply SMOTE on the training set only ====
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [ ]:
print("\n✅ After SMOTE balancing:")
print(y_train_res.value_counts())


✅ After SMOTE balancing:
health_risk_high
1    2792
0    2792
Name: count, dtype: int64


In [ ]:

# ==== Optional: check shapes ====
print("\nX_train before SMOTE:", X_train.shape)
print("X_train after SMOTE:", X_train_res.shape)


X_train before SMOTE: (3998, 16)
X_train after SMOTE: (5584, 16)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numeric_cols = X_train_res.select_dtypes(include=['float64', 'int64']).columns

# Fit on balanced train
scaler.fit(X_train_res[numeric_cols])

# Transform train and test
X_train_res[numeric_cols] = scaler.transform(X_train_res[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])


In [ ]:
df


,age,weight,height,exercise,sleep,sugar_intake,bmi_recalc,smoking_yes,alcohol_yes,health_risk_high,profession_doctor,profession_driver,profession_engineer,profession_farmer,profession_office_worker,profession_student,profession_teacher
0,56,67,195,1.0,6.1,1.0,17.619987,True,True,1,False,False,False,False,True,False,False
1,69,76,170,3.0,6.9,2.0,26.297578,False,False,1,False,False,False,False,False,False,True
2,46,106,153,3.0,6.6,0.0,45.281729,True,False,1,False,False,False,False,False,False,False
3,32,54,186,2.0,8.5,1.0,15.608741,False,False,0,False,False,False,False,False,False,False
4,60,98,195,3.0,8.0,0.0,25.772518,False,False,1,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,42,75,187,2.0,7.2,0.0,21.447568,False,False,0,False,True,False,False,False,False,False
4994,39,86,183,1.0,7.4,0.0,25.680074,False,False,1,False,False,True,False,False,False,False
4995,48,56,194,0.0,7.2,2.0,14.879371,True,True,1,False,False,True,False,False,False,False
4996,34,109,148,2.0,9.9,1.0,49.762600,False,False,0,True,False,False,False,False,False,False
